<a href="https://www.kaggle.com/code/sscarecrow/phowhisperforaic?scriptVersionId=259821234" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Libraries Install

In [1]:
!pip -q install "transformers>=4.41" "accelerate>=0.30" torch torchaudio soundfile librosa
!ffmpeg -version >/dev/null 2>&1 || (apt-get -y update && apt-get -y install ffmpeg)
!pip install -q transformers sentence-transformers torch accelerate einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.3 MB/s eta 0:00:00


# Libraries Import

In [2]:
import os, json, math, shutil, subprocess, gc, time, re
from pathlib import Path
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
import cv2


2025-09-03 18:00:58.256586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756922458.515074      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756922458.582974      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Helper functions

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
VIDEO_PATH = "/kaggle/input/aic-sample-test/videos/L21_V003.mp4"
SHOT_FILE  = "/kaggle/input/aic-sample-test/shots_boundary/L21_V003.txt"
WORK_DIR   = "/kaggle/working"
TMP_DIR    = os.path.join(WORK_DIR, "tmp_shots")
os.makedirs(TMP_DIR, exist_ok=True)
JSONL_PATH = os.path.join(WORK_DIR, "L21_V003_transcripts.jsonl")  # append-as-you-go (resume safe)
FINAL_JSON = os.path.join(WORK_DIR, "L21_V003_transcript_summary.json")
MODEL_ID = "vinai/PhoWhisper-large"
END_FRAME_IS_INCLUSIVE = True   
FALLBACK_MIN_DURATION_S = 0.01  
ASR_CHUNK_LENGTH_S = 30         
ASR_BATCH_SIZE = 16             
AUDIO_RATE = 16000              
AUDIO_CHANNELS = 1

In [4]:
def check_ffmpeg():
    try:
        out = subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if out.returncode == 0:
            print("ffmpeg available")
        else:
            print("ffmpeg not found in PATH; cutting may fail.")
    except FileNotFoundError:
        print("ffmpeg not installed. Please install or use an environment with ffmpeg.")

def get_fps(video_path: str) -> float:
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Failed to open video: {video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    if not fps or math.isclose(fps, 0.0):
        raise RuntimeError("Could not determine FPS from video.")
    return float(fps)

def read_shot_file(txt_path: str):
    shots = []
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = re.split(r"[,\s]+", line)
            if len(parts) < 2: 
                continue
            try:
                s, e = int(parts[0]), int(parts[1])
                if e < s:
                    s, e = e, s
                shots.append((s, e))
            except ValueError:
                continue
    return shots

def frames_to_times(start_f, end_f, fps: float, end_inclusive=True):
    if end_inclusive:
        duration = (end_f - start_f + 1) / fps
    else:
        duration = (end_f - start_f) / fps
    duration = max(duration, FALLBACK_MIN_DURATION_S)
    start_s = start_f / fps
    end_s = start_s + duration
    return start_s, duration, end_s

def ffmpeg_cut_video(in_path: str, out_path: str, start_s: float, duration_s: float, reencode=False):
    cmd = ["ffmpeg", "-y", "-ss", f"{start_s:.6f}", "-t", f"{duration_s:.6f}", "-i", in_path]
    if reencode:
        cmd += ["-c:v", "libx264", "-c:a", "aac", "-movflags", "+faststart"]
    else:
        cmd += ["-c", "copy"]
    cmd += [out_path]
    r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if r.returncode != 0:
        raise RuntimeError(f"ffmpeg cut failed: {r.stderr[:4000]}")

def ffmpeg_extract_wav(in_video: str, out_wav: str, sr=AUDIO_RATE, ch=AUDIO_CHANNELS):
    cmd = [
        "ffmpeg", "-y", "-i", in_video, "-vn",
        "-ac", str(ch), "-ar", str(sr), "-acodec", "pcm_s16le",
        out_wav
    ]
    r = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if r.returncode != 0:
        raise RuntimeError(f"ffmpeg audio extract failed: {r.stderr[:4000]}")

def safe_remove(path: str):
    try:
        if os.path.isfile(path):
            os.remove(path)
        elif os.path.isdir(path):
            shutil.rmtree(path)
    except Exception:
        pass

def load_existing_shot_ids(jsonl_path: str):
    done = set()
    if not os.path.exists(jsonl_path):
        return done
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rec = json.loads(line)
                done.add(rec.get("shot_index"))
            except Exception:
                continue
    return done


In [5]:
device = 0 if torch.cuda.is_available() else -1
dtype  = torch.float16 if torch.cuda.is_available() else torch.float32
processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    low_cpu_mem_usage=True
)
gen = model.generation_config
gen.forced_decoder_ids = processor.get_decoder_prompt_ids(language="vi", task="transcribe")
gen.language = "vi"
gen.task = "transcribe"
model.generation_config = gen
if device == 0:
    model = model.to("cuda")
asr = pipeline(
    task="automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=device
)
print("PhoWhisper loaded:", MODEL_ID)
check_ffmpeg()

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


PhoWhisper loaded: vinai/PhoWhisper-large
ffmpeg available


In [6]:
video_id = Path(VIDEO_PATH).stem  # "L21_V003"
fps = get_fps(VIDEO_PATH)
print(f"Video: {video_id} | FPS: {fps:.3f}")
shots = read_shot_file(SHOT_FILE)
print(f"Found {len(shots)} shots in {Path(SHOT_FILE).name}")
done_shots = load_existing_shot_ids(JSONL_PATH)
if done_shots:
    print(f"Resuming: {len(done_shots)} shots already processed.")
processed = 0
start_all = time.time()
with torch.inference_mode():
    for idx, (sf, ef) in enumerate(shots):
        if idx in done_shots:
            continue

        start_s, dur_s, end_s = frames_to_times(sf, ef, fps, END_FRAME_IS_INCLUSIVE)
        shot_mp4 = os.path.join(TMP_DIR, f"{video_id}_shot_{idx:04d}.mp4")
        shot_wav = os.path.join(TMP_DIR, f"{video_id}_shot_{idx:04d}.wav")

        try:
            ffmpeg_cut_video(VIDEO_PATH, shot_mp4, start_s, dur_s, reencode=False)
            ffmpeg_extract_wav(shot_mp4, shot_wav, sr=AUDIO_RATE, ch=AUDIO_CHANNELS)
            t0 = time.time()
            result = asr(
                shot_wav,
                chunk_length_s=ASR_CHUNK_LENGTH_S,
                batch_size=ASR_BATCH_SIZE
            )
            t1 = time.time()
            text = result.get("text", "").strip() if isinstance(result, dict) else str(result).strip()
            rec = {
                "video_id": video_id,
                "shot_index": idx,
                "start_frame": sf,
                "end_frame": ef,
                "fps": fps,
                "start_sec": round(start_s, 6),
                "end_sec": round(end_s, 6),
                "duration_sec": round(dur_s, 6),
                "transcript": text,
                "model_id": MODEL_ID,
                "processing_time_sec": round(t1 - t0, 3),
                "timestamp": int(time.time())
            }
            with open(JSONL_PATH, "a", encoding="utf-8") as f:
                f.write(json.dumps(rec, ensure_ascii=False) + "\n")
            processed += 1
            print(f"[{idx+1}/{len(shots)}] Shot {idx:04d} -> {dur_s:.2f}s, "
                  f"{'EMPTY' if not text else f'{len(text)} chars'} | {t1-t0:.1f}s")

        except Exception as e:
            err = {
                "video_id": video_id,
                "shot_index": idx,
                "start_frame": sf,
                "end_frame": ef,
                "fps": fps,
                "start_sec": round(start_s, 6),
                "end_sec": round(end_s, 6),
                "duration_sec": round(dur_s, 6),
                "error": f"{type(e).__name__}: {str(e)[:1000]}",
                "model_id": MODEL_ID,
                "timestamp": int(time.time())
            }
            with open(JSONL_PATH, "a", encoding="utf-8") as f:
                f.write(json.dumps(err, ensure_ascii=False) + "\n")
            print(f"⚠️ Shot {idx:04d} failed: {e}")

        finally:
            safe_remove(shot_mp4)
            safe_remove(shot_wav)
            del_result = None
            try:
                del result
            except Exception:
                pass
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

elapsed = time.time() - start_all
print(f"Done. Processed {processed}/{len(shots)} shots in {elapsed/60:.1f} min.")

Video: L21_V003 | FPS: 25.000
Found 253 shots in L21_V003.txt


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [(1, 50278), (2, 50359), (3, 50363)] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[1/253] Shot 0000 -> 11.92s, 159 chars | 6.2s


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


[2/253] Shot 0001 -> 2.52s, 39 chars | 1.5s


...

[251/253] Shot 0250 -> 6.60s, 105 chars | 2.9s


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


[252/253] Shot 0251 -> 15.80s, 298 chars | 6.7s


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


[253/253] Shot 0252 -> 4.28s, 67 chars | 2.1s
Done. Processed 253/253 shots in 12.7 min.


In [7]:
segments = []
with open(JSONL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try:
            j = json.loads(line)
        except Exception:
            continue
        if "error" in j:
            continue
        segments.append({
            "shot_index": j["shot_index"],
            "start_frame": j["start_frame"],
            "end_frame": j["end_frame"],
            "start_sec": j["start_sec"],
            "end_sec": j["end_sec"],
            "duration_sec": j["duration_sec"],
            "transcript": j.get("transcript", "")
        })

segments.sort(key=lambda x: x["shot_index"])

summary = {
    "video_id": video_id,
    "fps": fps,
    "num_shots": len(shots),
    "num_segments_transcribed": len(segments),
    "model_id": MODEL_ID,
    "segments": segments
}

with open(FINAL_JSON, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)
print(f"Saved summary JSON: {FINAL_JSON}")
print(f"Also kept streaming results (resume-safe) at: {JSONL_PATH}")


Saved summary JSON: /kaggle/working/L21_V003_transcript_summary.json
Also kept streaming results (resume-safe) at: /kaggle/working/L21_V003_transcripts.jsonl
